## Using SQL Express Developer Edition on local laptop

**Stage 1, import original selections**

In [ ]:
IF OBJECT_ID ('TEMPDB..#ORIGINAL_SELECTION') IS NOT NULL DROP TABLE #ORIGINAL_SELECTION
GO
CREATE TABLE #ORIGINAL_SELECTION
(	
	CUSTOMER_ID int,
	LINK VARCHAR(250)
);

BULK INSERT #ORIGINAL_SELECTION
FROM 'C:\Users\napl\Downloads\customer_ids_links.csv'
WITH (
	DATAFILETYPE = 'char',
	FIRSTROW = 2,
	FIELDTERMINATOR = ',',
	ROWTERMINATOR = '\n' );

SELECT TOP 10 * FROM #ORIGINAL_SELECTION

**We need the value from the LINK column where the key is RID and store it in the same table so we can compare.**

In [ ]:
ALTER TABLE #ORIGINAL_SELECTION ADD RID BIGINT;
GO
UPDATE #ORIGINAL_SELECTION SET RID = REPLACE(LINK, 'https://bb.survey.com/G1122NL/survey/my_survey?name=G1122NL&RID=','');
GO
SELECT TOP 10 * FROM #ORIGINAL_SELECTION

**Import winners file**

In [ ]:
IF OBJECT_ID ('TEMPDB..#WINNERS') IS NOT NULL DROP TABLE #WINNERS
CREATE TABLE #WINNERS (
	[ID]	BIGINT
);
GO
BULK INSERT #WINNERS
FROM 'C:\Users\napl\Downloads\winners.csv'
WITH (
	DATAFILETYPE = 'char',
	FIRSTROW = 2,
	FIELDTERMINATOR = ',',
	ROWTERMINATOR = '\n' );
GO
SELECT TOP 10 * FROM #WINNERS;

**Check winners ID in relation to the original selection, this selection reveals the winners from the selection**

**Combine into an new temporary table, assign an identity column that we can use later..**

In [ ]:
IF OBJECT_ID ('TEMPDB..#EXPORT_WINNERS') IS NOT NULL DROP TABLE #EXPORT_WINNERS
GO
CREATE TABLE #EXPORT_WINNERS (
    ID          INT IDENTITY(1,1) NOT NULL PRIMARY KEY,
    CUSTOMER_ID INT,
    CARDNUMBER  BIGINT NULL,
    PIN VARCHAR(4)
);
GO
INSERT INTO #EXPORT_WINNERS (CUSTOMER_ID)
SELECT  S.CUSTOMER_ID
FROM    #WINNERS W
JOIN    #ORIGINAL_SELECTION S ON W.ID = S.RID;
GO
SELECT TOP 10 * FROM #EXPORT_WINNERS

**Import giftcard file**

In [ ]:
IF OBJECT_ID ('TEMPDB..#GIFTCARDS') IS NOT NULL DROP TABLE #GIFTCARDS
CREATE TABLE #GIFTCARDS
(	
	
	CARDNUMBER  BIGINT,
	PIN         VARCHAR(20),
	VALUE       INT
);
GO
BULK INSERT #GIFTCARDS
FROM 'C:\Users\napl\Downloads\giftcards.csv'
WITH (
    DATAFILETYPE = 'char',
    FIRSTROW = 2,
    FIELDTERMINATOR = ',',
    ROWTERMINATOR = '\n' );
GO
SELECT TOP 10 * FROM #GIFTCARDS

**Add identity column to the giftcards table, from 1 tom 150.... with that we have an relational key with the export\_winners table.**

In [ ]:
ALTER TABLE #GIFTCARDS ADD ROWID INT IDENTITY(1,1) NOT NULL PRIMARY KEY;
GO

SELECT TOP 10 * FROM #GIFTCARDS

**Now we can join the winners and the giftcards to form 1 table, we update the corresponding values**

**Then every customer has been assigned an unique cardnumber and pincode**

In [ ]:
SELECT *
FROM    #EXPORT_WINNERS E
JOIN    #GIFTCARDS G ON E.ID = G.ROWID

GO

In [ ]:
-- MAKE IT FINAL
UPDATE E 
    SET  CARDNUMBER = G.CARDNUMBER,
        PIN = G.PIN
FROM    #EXPORT_WINNERS E
JOIN    #GIFTCARDS G ON E.ID = G.ROWID
GO

**Get the final table**

In [ ]:
SELECT CUSTOMER_ID, CARDNUMBER, PIN
FROM #EXPORT_WINNERS